In [1]:
import sys
sys.path.append('../../')
from pprint import pprint
import time
import pickle
import importlib
import numpy as np

In [2]:
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter
from lib_grover import make_hit_list
from libs_qrem import LeastNormFilter

In [3]:
nbit = 10 # change this value to get discretized result closer to analytical results # 3
#setting the number of shots and Grover operators.
max_iter = 10
number_grover_list = [1, 2, 4, 8, 16, 32, 64]  # list of number of Grover operators
shots = 8192
shots_list = [shots] * len(number_grover_list) # list of number of shots
if len(shots_list) != len(number_grover_list):
    raise Exception('The length of shots_list should be equal to the length eof number_grover_list.')

In [4]:
with open("pkls/raw_hist_list_list.pkl", "rb") as f:
    raw_hist_list_list = pickle.load(f)
with open("pkls/meas_fitters.pkl", "rb") as f:
    meas_fitters = pickle.load(f)

In [5]:
lnp_mitigator_list = []
for i in range(0, max_iter):
    lnp_mitigator_list.append(LeastNormFilter(nbit + 1, meas_fitters[i].cal_matrices))
    if i % 10 == 0:
        print("size", i, "finished")
print("length of lnp_mitigator_list: ", len(lnp_mitigator_list))

size 0 finished
length of lnp_mitigator_list:  10


In [6]:
def get_info(mitigator):
    return {"exact_one_norm_of_reduced_inv_A": mitigator.exact_one_norm_of_reduced_inv_A(),
            "mitigated_hist": mitigator.mitigated_hist(),
            "times": mitigator.times(),
            "mitigation_stddev": mitigator.mitigation_stddev(norm_type = "exact"),
            }

In [7]:
lnp_info_list_list = []
lnp_hit_list_list = []
for i, raw_hist_list in enumerate(raw_hist_list_list):
    lnp_info_list = []
    lnp_hist_list = []
    for raw_hist in raw_hist_list:
        lnp_hist_list.append( lnp_mitigator_list[i].apply(raw_hist) )
        lnp_info_list.append( get_info(lnp_mitigator_list[i]) )
    lnp_hit_list_list.append( make_hit_list(lnp_hist_list) )
    lnp_info_list_list.append( lnp_info_list )
    print("iter", i, "finished")

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
iter 7 finished
iter 8 finished
iter 9 finished


In [8]:
with open("pkls/lnp_info_list_list.pkl", "wb") as f:
    pickle.dump(lnp_info_list_list, f)
with open("pkls/lnp_hit_list_list.pkl", "wb") as f:
    pickle.dump(lnp_hit_list_list, f)